In [1]:
from collections import deque
import taichi as ti
import taichi.math as tm
from taichi import sin, cos, sqrt
ti.reset()
ti.init(arch = ti.gpu, fast_math=True)

[Taichi] version 1.7.4, llvm 15.0.7, commit b4b956fd, osx, python 3.10.19
[Taichi] Starting on arch=metal


[I 11/09/25 23:35:19.621 533001] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


Assuming the double pendulum has the same arm lengths $l$ and mass $m$ we can write it as a system of 4 (hamilton) equations of motion like so. Let's call the two angles $x,y$ and the two velocities $u,v$ and call $\hat g = g/l$
$$
\begin{align*}
x' &= u\\
y' &= v\\
u' &= \frac{3\hat g \sin x + \hat g \sin(x - 2y) + u^2 \sin(2x - 2y) + 2 v^2 \sin(x - y)}{\cos(2x - 2y) - 3}\\
v' &= \frac{2\hat g \sin y - 2\hat g \sin(2x - y) - v^2 \sin(2x - 2y) - 4 u^2\sin(x - y)}{\cos(2x - 2y) - 3}
\end{align*} 
$$
Now we can define $a = (x,y,u,v)$ and $f$ be the rught hand side of this equation. We therefore have
$$
a' = f(a),
$$
which is a great idea to try and solve for using Runge kutta. So we will implement that.

In [2]:
# Some tweaking is in order
scale   = 2
dimL    = (900,900)
dim     = (int(scale*dimL[0]),int(scale*dimL[1]))
g       = 0.1
h       = 1e-1
e       = 1e-3
dx      = (2.5*tm.pi/dim[0], 2.5*tm.pi/dim[1]) # (1e-2,1e-2)
maxdist = 100

# Taichi arrays that will store our vectors
vec     = tm.vec4
a       = ti.Vector.field(4, ti.f32, dim)
b       = ti.Vector.field(4, ti.f32, dim)
pixels  = ti.Vector.field(3, ti.f32, dim)
pixelsL = ti.Vector.field(3, ti.f32, dimL)
window  = ti.ui.Window("Double Pendulum", res=dimL, fps_limit=400)
gui     = window.get_canvas()

max = ti.field(ti.f32,())
T = ti.field(ti.f32, ())
C = ti.field(ti.i32, ())

In [ ]:
# Derivative function
@ti.func
def f(a:vec) -> vec:
    # Precalculate some sines ans cosines so that we don't do it twice
    sxy     = sin(  a[0] -   a[1])
    s2x2y   = sin(2*a[0] - 2*a[1])
    c2x2y   = cos(2*a[0] - 2*a[1]) - 3

    # Return the vector b = f(a)
    return  vec(a[2], \
                a[3], \
                (3 * g * sin(a[0]) +   g*sin(a[0] - 2*a[1]) + a[2]*a[2]*s2x2y + 2*a[3]*a[3]*sxy)/c2x2y, \
                (2 * g * sin(a[1]) - 2*g*sin(2*a[0] - a[1]) + a[3]*a[3]*s2x2y - 2*a[2]*a[2]*sxy)/c2x2y) 


# # Derivative function
# @ti.func
# def f(a:vec) -> vec:
#     # Precalculate some sines ans cosines so that we don't do it twice
#     sx  = sin(a[0])
#     sy  = sin(a[1])
#     sxy = sin(a[0] - a[1])
#     cxy = cos(a[0] - a[1])

#     # Return the vector b = f(a)
#     return  vec(a[2], 
#                 a[3], 
#                 (-g*(sx + sy*cxy) - a[3]**2 * sxy * (1 + cxy) + g * sy * cxy)/(1+sxy**2), 
#                 (a[2]**2 * sxy + g * sx * cxy - g * sy + a[3]**2 * sxy * cxy)/(1+sxy**2)) 


# Simple RK4 solver
@ti.func
def step(a:vec, h:ti.f32) -> vec:
    k1 = f(a)
    k2 = f(a + (h/2)*k1)
    k3 = f(a + (h/2)*k2)
    k4 = f(a + h*k3)
    
    return a + (k1 + 2*k2 + 2*k3 + k4)*(h/6)


# Intialization
@ti.kernel
def initialize(dx:float, dy:float, e:float, dimX:int, dimY:int):
    a.fill(0)
    b.fill(0)
    for i,j in a:
        a[i,j][0] = (i-dimX//2            )*dx
        a[i,j][1] = (j-dimY//2            )*dy
        b[i,j][0] = (i-dimX//2 + e/sqrt(2))*dx 
        b[i,j][1] = (j-dimY//2 + e/sqrt(2))*dy

# Draw the next frame
@ti.kernel
def draw(h:float, norm:float):
    max[None] = 0.0
    T[None] = 0.0
    C[None] = 0
    thr = 0.95
    for i,j in pixels:
        a[i,j] = step(a[i,j], h)
        b[i,j] = step(b[i,j], h)

        mag = (a[i,j][:2] - b[i,j][:2]).norm() # type: ignore
        ti.atomic_max(max[None], mag)
        pixels[i,j] = tm.vec3([mag,2*mag,5*mag])/(7*norm)
    
    for i, j in pixels:
        mag = (a[i, j][:2] - b[i, j][:2]).norm()
        if mag > max[None] * thr:
            ti.atomic_add(T[None], mag)
            ti.atomic_add(C[None], 1)
    
    # if C[None] > 0:
    max[None] = T[None] / C[None]

# Implement some antialiasing
@ti.kernel
def downsample():
    for i, j in pixelsL:
        acc = tm.vec3(0.0)
        for di, dj in ti.static(ti.ndrange(scale, scale)):
            acc += pixels[i * scale + di, j * scale + dj]
        pixelsL[i, j] = acc / (scale * scale)


In [4]:
initialize(*dx,e,*dim)

update_needed = True
steps   = 50
s       = 0
win     = 60
norms   = deque([1.]*win,maxlen=win)
norm    = 1.

def roll(new):
    global norm
    norm += (new - norms[0])/win
    norms.append(new if new == new else 1.0)
    
while window.running:
    if window.is_pressed(ti.GUI.ESCAPE): 
        ti.sync()
        window.destroy()
        break

    if update_needed:
        # print(norm)
        draw(h, 1)
        roll(max[None])
        # roll(estimate_top_fraction(0.1,max))
        downsample()

        # s+=1
        # if s == steps:
        #     downsample()
        #     s=0
            
    gui.set_image(pixelsL)

    window.show()
